Änderungen:
Programm holte immer wieder Daten aus erster Seite der Abfrage, weil an die next_url pro Schleifendurchlauf die ursprünglichen params angehängt wurden (die statisch auf Seite 1 der Abfrage blieben). Vorgeschlagener Fix: Seitenabfrage direkt in params hochzählen und pro Seite/Schleifendurchgang aus API-URL und params neue Abfrage-URL generieren. URL ist jetzt Konstante (nur die API-Adresse), an die Filter/Queries und Seitenangaben in Abfrageschleife jeweils angehängt werden - fetch_data(): Argumente dois, publication_years gelöscht, weil Output der Funktion; Argument URL gelöscht, weil s.o. Zweifelhaft: Alle tatsächlich nötigen Argumente der Funktion (URL, params) sind jetzt global vorgegeben (params dabei aber nicht konstant), während Auswahl der ausgegebenen Daten in fetch_data jeweils umdefiniert werden muss, ich bin mir nicht sicher, ob das so schon der beste Stil ist.  - pd.set_options: Zum Prüfen der Ergebnisse Ansicht Dataframe geändert, kürzt Ausgabe nicht und bricht nicht um.

In [15]:
import requests
import pandas as pd

# Define the API URL
URL = "https://api.datacite.org/dois"

# Define the query parameters
params = {
    "client-id" : "WUUA.CONFIDENT",
    #"query": "types.resourceTypeGeneral:Collection",
    "page[number]": 1,
    "page[size]": 10
}

# Initialize lists to store data
dois = []
publication_years = []
title = []


# Function to fetch data from API and append to lists
def fetch_data():
    response = requests.get(URL, params=params)    
    antwort = response.json()
    for item in antwort["data"]:
        attributes = item["attributes"]
        dois.append(attributes["doi"])
        publication_years.append(attributes["publicationYear"])
        title.append(attributes["titles"][0]["title"])
    # Look up URL of next page
    return antwort["links"].get("next")


# Fetch data ("while True:" to fetch all pages at once)
while params["page[number]"] < 3: 
    next_page = fetch_data()
    # Check if there is a next page to fetch, if yes, count up page number of query
    if next_page == None:
        break
    else: 
        params["page[number]"] += 1


In [16]:
# Create a DataFrame from the extracted data

# Show DataFrame untruncated
pd.set_option('display.max_rows', None)  
pd.set_option('display.max_columns', None)
pd.set_option('display.width',1000)

df = pd.DataFrame({
    "DOI": dois,
    "Publication Year": publication_years,
    "Title": title
})


# Generate a statistic of events per publication year
event_counts = df["Publication Year"].value_counts().sort_index()

# Print the statistic
print("Statistic of events per publication year:")
print(event_counts)

Statistic of events per publication year:
Publication Year
1997    2
1998    1
1999    1
2001    2
2002    1
2005    1
2006    2
2007    1
2008    1
2009    1
2010    1
2011    1
2012    1
2021    1
2022    1
2023    1
2024    1
Name: count, dtype: int64


In [17]:
print(df)

                   DOI  Publication Year                                              Title
0   10.25798/rsvj-2s07              2012                     Data Systems in Aerospace 2012
1   10.25798/jc7k-bg66              1997                          Data Systems in Aerospace
2   10.25798/jjz3-r706              2008                     Data Systems in Aerospace 2008
3   10.25798/28d1-9224              2009                     Data Systems in Aerospace 2009
4   10.25798/pnhh-wf36              2007                     Data Systems in Aerospace 2007
5   10.25798/hzfg-jk52              2010                     Data Systems in Aerospace 2010
6   10.25798/4bbx-tc78              2011                     Data Systems in Aerospace 2011
7   10.25798/94nq-zg90              2001                     Data Systems in Aerospace 2000
8   10.25798/504r-8926              2006                     Data Systems in Aerospace 2006
9   10.25798/fmjb-0279              2002                     Data Systems in Aer